In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC # Data Processing & Exploratory Data Analysis
# MAGIC 
# MAGIC This notebook:
# MAGIC - Loads data from Delta Bronze layer
# MAGIC - Identifies cancer cases using ICD-9 diagnosis codes
# MAGIC - Performs exploratory data analysis
# MAGIC - Cleans and prepares data for modeling
# MAGIC - Saves processed data to Delta Silver layer

# COMMAND ----------

# MAGIC %md
# MAGIC ## 1. Import Libraries and Configuration

# COMMAND ----------

# Import libraries
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import seaborn as sns

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# COMMAND ----------

# Configuration - Data Paths
BASE_PATH = "/Volumes/workspace/default/file_store"
DELTA_BASE_PATH = f"{BASE_PATH}/delta"
DELTA_BRONZE_PATH = f"{DELTA_BASE_PATH}/bronze"
DELTA_SILVER_PATH = f"{DELTA_BASE_PATH}/silver"
DELTA_GOLD_PATH = f"{DELTA_BASE_PATH}/gold"

print("✓ Configuration loaded")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 2. Load Data from Delta Bronze Layer

# COMMAND ----------

# Load beneficiary data
beneficiary_df = spark.read.format("delta").load(f"{DELTA_BRONZE_PATH}/beneficiary")
print(f"Beneficiary records: {beneficiary_df.count():,}")

# Load inpatient claims
inpatient_df = spark.read.format("delta").load(f"{DELTA_BRONZE_PATH}/inpatient_claims")
print(f"Inpatient claims: {inpatient_df.count():,}")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 3. Explore Diagnosis Codes
# MAGIC 
# MAGIC CMS data uses ICD-9 codes (2008-2010 data). Cancer codes typically start with:
# MAGIC - **140-239**: Neoplasms
# MAGIC - **140-208**: Malignant neoplasms
# MAGIC - **210-229**: Benign neoplasms
# MAGIC - **230-234**: Carcinoma in situ

# COMMAND ----------

# Show available diagnosis code columns
diag_columns = [col for col in inpatient_df.columns if 'ICD9_DGNS_CD' in col or 'ADMTNG_ICD9_DGNS_CD' in col]
print(f"Diagnosis code columns found: {len(diag_columns)}")
print(diag_columns[:10])  # Show first 10

# COMMAND ----------

# Sample diagnosis codes
display(inpatient_df.select('DESYNPUF_ID', 'CLM_ID', 'ADMTNG_ICD9_DGNS_CD', 
                            'ICD9_DGNS_CD_1', 'ICD9_DGNS_CD_2', 'ICD9_DGNS_CD_3').limit(20))

# COMMAND ----------

# MAGIC %md
# MAGIC ## 4. Define Cancer ICD-9 Codes

# COMMAND ----------

# Function to check if a diagnosis code is cancer-related
def is_cancer_code(code):
    """
    Check if ICD-9 code is cancer-related
    Cancer codes are in ranges: 140-208 (malignant), 210-229 (benign), 230-234 (in situ)
    """
    if code is None:
        return False
    
    # Remove decimal point and convert to string
    code_str = str(code).replace('.', '')
    
    try:
        # Extract numeric part (first 3 digits)
        code_num = int(code_str[:3])
        
        # Check if in cancer ranges
        if 140 <= code_num <= 239:
            return True
    except:
        pass
    
    return False

# Register UDF
is_cancer_udf = udf(is_cancer_code, BooleanType())

# COMMAND ----------

# MAGIC %md
# MAGIC ## 5. Identify Cancer Cases

# COMMAND ----------

# Create a list of all diagnosis columns to check
diag_check_columns = ['ADMTNG_ICD9_DGNS_CD'] + [f'ICD9_DGNS_CD_{i}' for i in range(1, 11)]

# Create boolean columns for each diagnosis field
for col_name in diag_check_columns:
    if col_name in inpatient_df.columns:
        inpatient_df = inpatient_df.withColumn(
            f"{col_name}_is_cancer",
            is_cancer_udf(col(col_name))
        )

# COMMAND ----------

# Identify claims with at least one cancer diagnosis
# Create a single flag: has_cancer_diagnosis using OR logic
# Start with False, then OR all the cancer flag columns
has_cancer_expr = lit(False)
for cancer_col in cancer_flag_cols:
    has_cancer_expr = has_cancer_expr | coalesce(col(cancer_col), lit(False))

inpatient_df = inpatient_df.withColumn(
    'has_cancer_diagnosis',
    has_cancer_expr
)

# COMMAND ----------

# Count cancer cases
cancer_claims = inpatient_df.filter(col('has_cancer_diagnosis') == True)
total_cancer_claims = cancer_claims.count()
total_claims = inpatient_df.count()

print(f"Total inpatient claims: {total_claims:,}")
print(f"Cancer-related claims: {total_cancer_claims:,}")
print(f"Percentage: {(total_cancer_claims/total_claims)*100:.2f}%")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 6. Extract Primary Cancer Diagnosis

# COMMAND ----------

# Function to extract first cancer diagnosis code
def extract_primary_cancer_code(row):
    """Extract the first cancer diagnosis code found in the claim"""
    for col_name in diag_check_columns:
        if col_name in row and row[col_name] and is_cancer_code(row[col_name]):
            return str(row[col_name])
    return None

# Create temp view to use SQL
cancer_claims.createOrReplaceTempView("cancer_claims_temp")

# COMMAND ----------

# Extract primary cancer codes
cancer_with_primary = cancer_claims.withColumn(
    'primary_cancer_code',
    when(col('ADMTNG_ICD9_DGNS_CD_is_cancer'), col('ADMTNG_ICD9_DGNS_CD'))
    .when(col('ICD9_DGNS_CD_1_is_cancer'), col('ICD9_DGNS_CD_1'))
    .when(col('ICD9_DGNS_CD_2_is_cancer'), col('ICD9_DGNS_CD_2'))
    .when(col('ICD9_DGNS_CD_3_is_cancer'), col('ICD9_DGNS_CD_3'))
    .otherwise(lit(None))
)

# COMMAND ----------

# Show distribution of cancer diagnosis codes
print("Top 20 Cancer Diagnosis Codes:")
cancer_code_dist = cancer_with_primary.groupBy('primary_cancer_code') \
    .agg(count('*').alias('claim_count')) \
    .orderBy(desc('claim_count')) \
    .limit(20)

display(cancer_code_dist)

# COMMAND ----------

# MAGIC %md
# MAGIC ## 7. Categorize Cancer Types

# COMMAND ----------

# Create cancer type categories based on ICD-9 ranges
def categorize_cancer_type(code):
    """Categorize cancer by ICD-9 code range"""
    if code is None:
        return 'Unknown'
    
    code_str = str(code).replace('.', '')
    try:
        code_num = int(code_str[:3])
        
        if 140 <= code_num <= 149:
            return 'Lip, Oral Cavity, Pharynx'
        elif 150 <= code_num <= 159:
            return 'Digestive Organs'
        elif 160 <= code_num <= 165:
            return 'Respiratory System'
        elif 170 <= code_num <= 176:
            return 'Bone, Connective Tissue, Skin'
        elif 174 <= code_num <= 175:
            return 'Breast'
        elif 179 <= code_num <= 189:
            return 'Genitourinary Organs'
        elif 190 <= code_num <= 199:
            return 'Other and Unspecified Sites'
        elif 200 <= code_num <= 208:
            return 'Lymphatic and Hematopoietic'
        elif 210 <= code_num <= 229:
            return 'Benign Neoplasms'
        elif 230 <= code_num <= 234:
            return 'Carcinoma in Situ'
        else:
            return 'Other'
    except:
        return 'Unknown'

categorize_cancer_udf = udf(categorize_cancer_type, StringType())

cancer_with_primary = cancer_with_primary.withColumn(
    'cancer_type_category',
    categorize_cancer_udf(col('primary_cancer_code'))
)

# COMMAND ----------

# Show cancer type distribution
print("Cancer Type Distribution:")
cancer_type_dist = cancer_with_primary.groupBy('cancer_type_category') \
    .agg(count('*').alias('claim_count')) \
    .orderBy(desc('claim_count'))

display(cancer_type_dist)

# COMMAND ----------

# MAGIC %md
# MAGIC ## 8. Join with Beneficiary Data

# COMMAND ----------

# Join cancer claims with patient demographics
cancer_patient_df = cancer_with_primary.join(
    beneficiary_df,
    on='DESYNPUF_ID',
    how='inner'
)

print(f"Cancer patients with demographics: {cancer_patient_df.count():,}")

# COMMAND ----------

# Show sample of joined data
display(cancer_patient_df.select(
    'DESYNPUF_ID',
    'CLM_ID',
    'primary_cancer_code',
    'cancer_type_category',
    'BENE_BIRTH_DT',
    'BENE_SEX_IDENT_CD',
    'BENE_RACE_CD',
    'CLM_ADMSN_DT',
    'CLM_FROM_DT',
    'CLM_THRU_DT'
).limit(10))

# COMMAND ----------

# MAGIC %md
# MAGIC ## 9. Feature Engineering - Basic Demographics
# COMMAND ----------
# COMMAND ----------

# MAGIC %md
# MAGIC ## 9. Feature Engineering - Basic Demographics

# COMMAND ----------

# Convert date columns from integer (YYYYMMDD) to actual date type
cancer_patient_df = cancer_patient_df.withColumn(
    'birth_date',
    to_date(col('BENE_BIRTH_DT').cast('string'), 'yyyyMMdd')
)

cancer_patient_df = cancer_patient_df.withColumn(
    'admission_date',
    to_date(col('CLM_ADMSN_DT').cast('string'), 'yyyyMMdd')
)

# Calculate age at admission
cancer_patient_df = cancer_patient_df.withColumn(
    'age_at_admission',
    floor(datediff(col('admission_date'), col('birth_date')) / 365.25)
)

# Create age groups
cancer_patient_df = cancer_patient_df.withColumn(
    'age_group',
    when(col('age_at_admission') < 50, '<50')
    .when((col('age_at_admission') >= 50) & (col('age_at_admission') < 65), '50-64')
    .when((col('age_at_admission') >= 65) & (col('age_at_admission') < 75), '65-74')
    .when(col('age_at_admission') >= 75, '75+')
    .otherwise('Unknown')
)

# Map gender codes
cancer_patient_df = cancer_patient_df.withColumn(
    'gender',
    when(col('BENE_SEX_IDENT_CD') == 1, 'Male')
    .when(col('BENE_SEX_IDENT_CD') == 2, 'Female')
    .otherwise('Unknown')
)

# COMMAND ----------

# MAGIC %md
# MAGIC ## 10. Exploratory Data Analysis

# COMMAND ----------

# Age distribution
print("=== Age Distribution ===")
age_dist = cancer_patient_df.groupBy('age_group') \
    .agg(count('*').alias('count')) \
    .orderBy('age_group')
display(age_dist)

# COMMAND ----------

# Gender distribution
print("=== Gender Distribution ===")
gender_dist = cancer_patient_df.groupBy('gender') \
    .agg(count('*').alias('count'))
display(gender_dist)

# COMMAND ----------

# Cancer type by gender
print("=== Cancer Type by Gender ===")
cancer_gender = cancer_patient_df.groupBy('cancer_type_category', 'gender') \
    .agg(count('*').alias('count')) \
    .orderBy(desc('count'))
display(cancer_gender.limit(20))

# COMMAND ----------

# Cancer type by age group
print("=== Cancer Type by Age Group ===")
cancer_age = cancer_patient_df.groupBy('cancer_type_category', 'age_group') \
    .agg(count('*').alias('count')) \
    .orderBy(desc('count'))
display(cancer_age.limit(20))

# COMMAND ----------

# MAGIC %md
# MAGIC ## 11. Visualizations

# COMMAND ----------

# Convert to Pandas for visualization
cancer_type_pd = cancer_type_dist.toPandas()

# Plot cancer type distribution
plt.figure(figsize=(12, 6))
plt.bar(range(len(cancer_type_pd)), cancer_type_pd['claim_count'])
plt.xticks(range(len(cancer_type_pd)), cancer_type_pd['cancer_type_category'], rotation=45, ha='right')
plt.xlabel('Cancer Type')
plt.ylabel('Number of Claims')
plt.title('Distribution of Cancer Types in CMS Medicare Data')
plt.tight_layout()
plt.show()

# COMMAND ----------

# Age distribution visualization
age_dist_pd = age_dist.toPandas()

plt.figure(figsize=(10, 6))
plt.bar(age_dist_pd['age_group'], age_dist_pd['count'], color='steelblue')
plt.xlabel('Age Group')
plt.ylabel('Number of Claims')
plt.title('Cancer Claims by Age Group')
plt.tight_layout()
plt.show()

# COMMAND ----------

# MAGIC %md
# MAGIC ## 12. Data Quality Check

# COMMAND ----------

# Check for missing values in key fields
print("=== Missing Values in Key Fields ===")
key_fields = ['DESYNPUF_ID', 'primary_cancer_code', 'cancer_type_category', 
              'age_at_admission', 'gender', 'CLM_ADMSN_DT']

for field in key_fields:
    null_count = cancer_patient_df.filter(col(field).isNull()).count()
    total_count = cancer_patient_df.count()
    pct_missing = (null_count / total_count) * 100
    print(f"{field}: {null_count:,} missing ({pct_missing:.2f}%)")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 13. Save to Delta Silver Layer

# COMMAND ----------

# Select relevant columns for Silver layer
silver_df = cancer_patient_df.select(
    'DESYNPUF_ID',
    'CLM_ID',
    'primary_cancer_code',
    'cancer_type_category',
    'age_at_admission',
    'age_group',
    'gender',
    'BENE_RACE_CD',
    'CLM_ADMSN_DT',
    'CLM_FROM_DT',
    'CLM_THRU_DT',
    'CLM_PMT_AMT',
    'NCH_PRMRY_PYR_CLM_PD_AMT',
    'ADMTNG_ICD9_DGNS_CD',
    'ICD9_DGNS_CD_1',
    'ICD9_DGNS_CD_2',
    'ICD9_DGNS_CD_3'
)

# COMMAND ----------

# Save to Delta Silver layer
silver_df.write \
    .format("delta") \
    .mode("overwrite") \
    .save(f"{DELTA_SILVER_PATH}/cancer_claims")

print(f"✓ Cancer claims saved to Silver layer: {silver_df.count():,} records")

# COMMAND ----------

# Verify saved data
silver_verify = spark.read.format("delta").load(f"{DELTA_SILVER_PATH}/cancer_claims")
print(f"✓ Verified Silver layer: {silver_verify.count():,} records")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Summary
# MAGIC 
# MAGIC ✓ Identified cancer cases using ICD-9 codes  
# MAGIC ✓ Categorized cancer types  
# MAGIC ✓ Joined with patient demographics  
# MAGIC ✓ Created age and gender features  
# MAGIC ✓ Performed exploratory data analysis  
# MAGIC ✓ Saved cleaned data to Delta Silver layer  
# MAGIC 
# MAGIC **Next Steps:**
# MAGIC - Feature engineering for ML models (03_feature_engineering)
# MAGIC - Build predictive models (04_model_training)